In [9]:
import json
from transformers import AutoTokenizer, AutoModel
import torch

In [4]:
sentences_file = open("data/sentences.json")
sentences = json.load(sentences_file)

In [5]:
sentences

['A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
 'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
 'Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'As of 2018, approximately 37.9 million people are infected with HIV globally.',
 'Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.',
 'Classic cholera symptom is large amounts of watery diarrhea that lasts a few days. Vomiting and muscle cramps may also occur. Diarrhea can be so severe that it leads within hours to severe dehydration and electrolyte imbalance.',
 'The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome 

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
vectors = [
  model(**tokenizer(sentence, return_tensors="pt"))[0].detach().squeeze()
  for sentence in sentences
]

In [8]:
[v.size() for v in vectors]

[torch.Size([35, 768]),
 torch.Size([37, 768]),
 torch.Size([25, 768]),
 torch.Size([18, 768]),
 torch.Size([24, 768]),
 torch.Size([55, 768]),
 torch.Size([57, 768]),
 torch.Size([24, 768]),
 torch.Size([27, 768]),
 torch.Size([35, 768]),
 torch.Size([43, 768])]

In [10]:
averaged_vectors = [torch.mean(vector, dim=0) for vector in vectors]

[v.size() for v in averaged_vectors]

[torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768])]

In [11]:
def encode(document: str) -> torch.Tensor:
  tokens = tokenizer(document, return_tensors="pt")
  vector = model(**tokens)[0].detach().squeeze()
  return torch.mean(vector, dim=0)

In [14]:
import faiss
import numpy as np

index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(
  np.array([t.numpy() for t in averaged_vectors]),
  np.array(range(0, len(sentences)))
)

def search(query: str, k=1):
  encoded_query = encode(query).unsqueeze(dim=0).numpy()
  top_k = index.search(encoded_query, k)
  scores = top_k[0][0]
  results = [sentences[_id] for _id in top_k[1][0]]
  return list(zip(results, scores))

In [15]:
sentences[1]

'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.'

In [17]:
search(sentences[1], k=5)

[('The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
  72.59782),
 ('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
  65.67766),
 ('The death toll of Spanish Flu is estimated to have been somewhere between 17 million and 50 million, and possibly as high as 100 million, making it one of the deadliest pandemics in human history.',
  65.32672),
 ('The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.',
  64.27381),
 ('As of 2018, approximately 37.9 million people are infected with HIV globally.',
  61.73901)]

In [18]:
search("Spanish flu", k=5)

[('The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.',
  46.58074),
 ('Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.',
  40.58732),
 ('Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.', 39.97648),
 ('As of 2018, approximately 37.9 million people are infected with HIV globally.',
  39.54162),
 ('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
  38.532223)]